### Scenario 1 - Data (De)anonymization for enhanced privacy

Visit [webpage](https://pndang.com/GenAI_Capstone_ProjectOne/) for more details.

Test prompt: 1 - 
""" 
    Please translate the following to Vietnamese: 

    "Hi Robert! Please see below for your requested info

    full name: Phu Dang
    SSN: 123-45-6789
    email: pndang@ucsd.edu
    birthday: 04/11/2003, so I am 21 years old
    What's your nationality? I am a Vietnamese citizen
    What's your credit score? 775 FICO
    What's the loan amount requested? $1,000,000
    What's the loan purpose? Invest $500,000 in Sustana Cooling Partners as working capital and cash out the rest for my hi-performing restaurant business (Beers & Bastards, LLC). 

    Please ping me the best terms you could offer. Thanks!" 

"""

#### Private AI is up next!

In [2]:
from privateai_client import PAIClient
from privateai_client import request_objects
from openai import OpenAI 
from dotenv import load_dotenv
import os

load_dotenv()


True

In [3]:
# Set up configurations

PRIVATEAI_API_KEY = os.getenv("PRIVATEAI_API_KEY")
PRIVATEAI_URL = os.getenv("PRIVATEAI_URL")

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [4]:
prompt = """
    Hi Robert! Please see below for your requested info

    full name: Phu Dang
    SSN: 123-45-6789
    email: pndang@ucsd.edu
    birthday: 04/11/2003, so I am 21 years old
    What's your nationality? I am a Vietnamese citizen
    What's your credit score? 775 FICO
    What's the loan amount requested? $1,000,000
    What's the loan purpose? Invest $500,000 in Sustana Cooling Partners as working capital and cash out the rest for my hi-performing restaurant business (Beers & Bastards, LLC). 

    Please ping me the best terms you could offer. Thanks!"
    """

In [5]:
# quick demo

client = PAIClient(url="https://api.private-ai.com/community", api_key=PRIVATEAI_API_KEY)
text_request = request_objects.process_text_obj(text=[prompt])
response = client.process_text(text_request)
                    
print(text_request.text[0])
print()
print(response.processed_text[0])


    Hi Robert! Please see below for your requested info

    full name: Phu Dang
    SSN: 123-45-6789
    email: pndang@ucsd.edu
    birthday: 04/11/2003, so I am 21 years old
    What's your nationality? I am a Vietnamese citizen
    What's your credit score? 775 FICO
    What's the loan amount requested? $1,000,000
    What's the loan purpose? Invest $500,000 in Sustana Cooling Partners as working capital and cash out the rest for my hi-performing restaurant business (Beers & Bastards, LLC). 

    Please ping me the best terms you could offer. Thanks!"
    


    Hi [NAME_GIVEN_1]! Please see below for your requested info

    full name: [NAME_1]
    SSN: [SSN_1]
    email: [EMAIL_ADDRESS_1]
    birthday: [DOB_1], so I am [AGE_1] years old
    What's your nationality? I am a [ORIGIN_1] citizen
    What's your credit score? [NUMERICAL_PII_1] [ORGANIZATION_1]
    What's the loan amount requested? [MONEY_1]
    What's the loan purpose? Invest [MONEY_2] in [ORGANIZATION_2] as working ca

In [6]:
# build local entities map 
completions = {}

completions["raw_text"] = prompt
redaction_request_obj = request_objects.process_text_obj(text=[prompt])
redaction_response_obj = client.process_text(redaction_request_obj)

# store redactions
deidentified_text = redaction_response_obj.processed_text[0]
completions["redacted_text"] = deidentified_text
entity_list = redaction_response_obj.get_reidentify_entities()

In [ ]:
# send redacted prompt to LLM 
GPT_MODEL = 'gpt-4'

res = OpenAI().chat.cosystem', 'content': 'You are a helpful English-Vietnamese translation assistant.'},
        {'role': 'user', 'content': deidentified_text}
    ]
)

res_text = res.choices[0].message.content
completions['redacted'] = res_text
print(res_text)

Xin chào [NAME_GIVEN_1]! Dưới đây là thông tin bạn yêu cầu:

Họ và tên: [NAME_1]
Số SSN: [SSN_1]
Email: [EMAIL_ADDRESS_1]
Ngày sinh: [DOB_1], vì vậy tôi [AGE_1] tuổi
Quốc tịch của bạn là gì? Tôi là công dân của [ORIGIN_1]
Điểm tín dụng của bạn là bao nhiêu? [NUMERICAL_PII_1] [ORGANIZATION_1]
Số tiền vay yêu cầu là bao nhiêu? [MONEY_1]
Mục đích vay tiền là gì? Đầu tư [MONEY_2] vào [ORGANIZATION_2] làm vốn lưu động và rút tiền mặt phần còn lại cho doanh nghiệp nhà hàng hiệu suất cao của tôi ([ORGANIZATION_3]).

Vui lòng ping tôi với các điều khoản tốt nhất bạn có thể cung cấp. Cảm ơn!"


In [8]:
# re-identify the anonymous LLM response

reidentification_request_obj = request_objects.reidentify_text_obj(
    processed_text=[completions["redacted"]], entities=entity_list)

reidentification_response_obj = client.reidentify_text(
    reidentification_request_obj)

completions["reidentified"] = reidentification_response_obj.body[0]

In [9]:
# voila

print(completions['reidentified'])

Xin chào Robert! Dưới đây là thông tin bạn yêu cầu:

Họ và tên: Phu Dang
Số SSN: 123-45-6789
Email: pndang@ucsd.edu
Ngày sinh: 04/11/2003, vì vậy tôi 21 tuổi
Quốc tịch của bạn là gì? Tôi là công dân của Vietnamese
Điểm tín dụng của bạn là bao nhiêu? 775 FICO
Số tiền vay yêu cầu là bao nhiêu? $1,000,000
Mục đích vay tiền là gì? Đầu tư $500,000 vào Sustana Cooling Partners làm vốn lưu động và rút tiền mặt phần còn lại cho doanh nghiệp nhà hàng hiệu suất cao của tôi (Beers & Bastards, LLC).

Vui lòng ping tôi với các điều khoản tốt nhất bạn có thể cung cấp. Cảm ơn!"


_________________________

Test prompt: 2 - 

"""
    Please translate the following to Vietnamese:

    "Dear Vietcombank, please help me deactivate the following credit card I lost:

    card #: 1234 5678 9101 1121
    card name: Phu Dang
    security code: 555

    please call me back at 669-274-8956 to confirm. Thanks"

"""

In [16]:
# wrapper function

def private_ai_pipeline(prompt):

    # build local entities map 
    completions = {}

    completions["raw_text"] = prompt
    redaction_request_obj = request_objects.process_text_obj(text=[prompt])
    redaction_response_obj = client.process_text(redaction_request_obj)

    # store redactions
    deidentified_text = redaction_response_obj.processed_text[0]
    completions["redacted_text"] = deidentified_text
    entity_list = redaction_response_obj.get_reidentify_entities()

    print(deidentified_text)

    # send redacted prompt to LLM 
    GPT_MODEL = 'gpt-4'
    res = OpenAI().chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {'role': 'system', 'content': 'You are a helpful English-Vietnamese translation assistant.'},
            {'role': 'user', 'content': deidentified_text}
        ]
    )

    res_text = res.choices[0].message.content
    completions['redacted'] = res_text
    print(res_text+'\n')

    # re-identify the anonymous LLM response

    reidentification_request_obj = request_objects.reidentify_text_obj(
        processed_text=[completions["redacted"]], entities=entity_list)

    reidentification_response_obj = client.reidentify_text(
        reidentification_request_obj)

    completions["reidentified"] = reidentification_response_obj.body[0]

    print(completions['reidentified'])

    return {
        "redacted prompt": deidentified_text, 
        "llm output": res_text, 
        "re-identified": completions['reidentified']
    }

In [17]:
prompt_2 = """Please translate the following to Vietnamese: 

    "Dear Vietcombank, please help me deactivate the following credit card I lost:

    card #: 1234 5678 9101 1121
    card name: Phu Dang
    security code: 555

    please call me back at 669-274-8956 to confirm. Thanks" 
    
    """

In [18]:
prompt2_output = private_ai_pipeline(prompt_2)

Please translate the following to [LANGUAGE_1]: 

    "Dear [ORGANIZATION_1], please help me deactivate the following credit card I lost:

    card #: [CREDIT_CARD_1]
    card name: [NAME_1]
    security code: [CVV_1]

    please call me back at [PHONE_NUMBER_1] to confirm. Thanks" 
    
    
"Kính thưa [ORGANIZATION_1], xin hãy giúp tôi hủy bỏ chiếc thẻ tín dụng sau đây mà tôi đã mất:

    số thẻ: [CREDIT_CARD_1]
    tên trên thẻ: [NAME_1]
    mã bảo mật: [CVV_1]

    xin hãy gọi lại cho tôi qua số [PHONE_NUMBER_1] để xác nhận. Cảm ơn"

"Kính thưa Vietcombank, xin hãy giúp tôi hủy bỏ chiếc thẻ tín dụng sau đây mà tôi đã mất:

    số thẻ: 1234 5678 9101 1121
    tên trên thẻ: Phu Dang
    mã bảo mật: 555

    xin hãy gọi lại cho tôi qua số 669-274-8956 để xác nhận. Cảm ơn"


__________________

Prompt 3 - """

    Please translate the following letter of intent to English:

    "Kính gửi Jeff,

    Công ty Triton Capital Partners (TCP), LLC, trân trọng bày tỏ mong muốn và ý định mua lại tài sản Framework Apartments tại 805 13th St, San Diego, CA 92101 với mức giá đề xuất là 200 triệu đô la Mỹ (USD 200,000,000), tùy thuộc vào các điều khoản và điều kiện cụ thể được nêu trong quá trình thương lượng và ký kết hợp đồng mua bán chính thức.

    Đại diện từ TCP sẽ liên hệ về các điều khoản giao dịch chi tiết sau. Chúng tôi mong muốn có cơ hội hợp tác với Quý vị trong giao dịch này. 

    Trân trọng,
    Triton Capital Partners, LLC"

"""

In [19]:
prompt_3 = """Please translate the following letter of intent to English:

    "Kính gửi Jeff,

    Công ty Triton Capital Partners (TCP), LLC, trân trọng bày tỏ mong muốn và ý định mua lại tài sản Framework Apartments tại 805 13th St, San Diego, CA 92101 với mức giá đề xuất là 200 triệu đô la Mỹ (USD 200,000,000), tùy thuộc vào các điều khoản và điều kiện cụ thể được nêu trong quá trình thương lượng và ký kết hợp đồng mua bán chính thức.

    Đại diện từ TCP sẽ liên hệ về các điều khoản giao dịch chi tiết sau. Chúng tôi mong muốn có cơ hội hợp tác với Quý vị trong giao dịch này. 

    Trân trọng,
    Triton Capital Partners, LLC"

"""

In [20]:
prompt3_output = private_ai_pipeline(prompt_3)

Please translate the following letter of intent to [LANGUAGE_1]:

    "Kính gửi [NAME_GIVEN_1],

    Công ty [ORGANIZATION_1], trân trọng bày tỏ mong muốn và ý định mua lại tài sản [LOCATION_ADDRESS_STREET_1] tại [LOCATION_ADDRESS_1] với mức giá đề xuất là [MONEY_1] ([MONEY_2]), tùy thuộc vào các điều khoản và điều kiện cụ thể được nêu trong quá trình thương lượng và ký kết hợp đồng mua bán chính thức.

    Đại diện từ [ORGANIZATION_2] sẽ liên hệ về các điều khoản giao dịch chi tiết sau. Chúng tôi mong muốn có cơ hội hợp tác với Quý vị trong giao dịch này. 

    Trân trọng,
    [ORGANIZATION_3]"


"Dear [NAME_GIVEN_1],

Công ty [ORGANIZATION_1] respectfully expresses its desire and intention to acquire the property at [LOCATION_ADDRESS_STREET_1] in [LOCATION_ADDRESS_1] with a proposed price of [MONEY_1] ([MONEY_2]), depending on the specific terms and conditions stated in the negotiation and contract signing process.

A representative from [ORGANIZATION_2] will contact you regarding de

In [23]:
print(f"anonymized prompt:\n{prompt3_output['redacted prompt']}")
print(f"anonymized llm response:\n{prompt3_output['llm output']}\n")
print(f"re-identified response:\n{prompt3_output['re-identified']}\n")

anonymized prompt:
Please translate the following letter of intent to [LANGUAGE_1]:

    "Kính gửi [NAME_GIVEN_1],

    Công ty [ORGANIZATION_1], trân trọng bày tỏ mong muốn và ý định mua lại tài sản [LOCATION_ADDRESS_STREET_1] tại [LOCATION_ADDRESS_1] với mức giá đề xuất là [MONEY_1] ([MONEY_2]), tùy thuộc vào các điều khoản và điều kiện cụ thể được nêu trong quá trình thương lượng và ký kết hợp đồng mua bán chính thức.

    Đại diện từ [ORGANIZATION_2] sẽ liên hệ về các điều khoản giao dịch chi tiết sau. Chúng tôi mong muốn có cơ hội hợp tác với Quý vị trong giao dịch này. 

    Trân trọng,
    [ORGANIZATION_3]"


anonymized llm response:
"Dear [NAME_GIVEN_1],

Công ty [ORGANIZATION_1] respectfully expresses its desire and intention to acquire the property at [LOCATION_ADDRESS_STREET_1] in [LOCATION_ADDRESS_1] with a proposed price of [MONEY_1] ([MONEY_2]), depending on the specific terms and conditions stated in the negotiation and contract signing process.

A representative from [O